# LangChain Agents Tutorial

## What are Agents?

Agents are LangChain's way of giving LLMs the ability to use tools and take actions. Unlike chains that follow a predetermined sequence, agents can decide which tools to use and in what order based on the user's input.

### Key Concepts:
- **Agent**: The core logic that decides what action to take
- **Tools**: Functions that the agent can use to interact with the world
- **Agent Executor**: The runtime that calls the agent and executes tools
- **Memory**: Optional component to remember past interactions

In this tutorial, we'll explore:
1. Basic Agent Setup
2. Custom Tools
3. Different Agent Types
4. Agent with Memory
5. Practical Examples

## 1. Setup and Installation

First, let's install the required packages:

In [37]:
# Install required packages
# !pip install langchain langchain-openai langchain-community wikipedia duckduckgo-search

**Note:** If you haven't installed the required packages yet, uncomment the line above and run it, or install from the requirements.txt file:
```bash
pip install -r ../requirements.txt
```

### Import Libraries and Setup API Keys

In [38]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI  # Changed to Azure
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Load environment variables
load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")

# Use AzureChatOpenAI for Azure OpenAI
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_version=AZURE_OPENAI_API_VERSION,
    api_key=AZURE_OPENAI_KEY,
    temperature=0
)

print("✓ Setup complete!")

✓ Setup complete!


## 2. Creating Custom Tools

Tools are functions that agents can use. Let's create some custom tools:

In [39]:
from langchain_core.tools import tool, Tool

# Method 1: Using @tool decorator
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def multiply_numbers(a: float, b: float) -> float:
    """Multiplies two numbers together."""
    return a * b

# Method 2: Using Tool class
def add_numbers(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

addition_tool = Tool(
    name="Addition",
    func=lambda x: eval(x),  # Note: eval is dangerous in production!
    description="Useful for adding numbers. Input should be in the format 'a + b'"
)

# Test the tools
print(f"Word length tool: '{get_word_length.name}' - {get_word_length.description}")
print(f"Multiply tool: '{multiply_numbers.name}' - {multiply_numbers.description}")
print(f"Addition tool: '{addition_tool.name}' - {addition_tool.description}")

Word length tool: 'get_word_length' - Returns the length of a word.
Multiply tool: 'multiply_numbers' - Multiplies two numbers together.
Addition tool: 'Addition' - Useful for adding numbers. Input should be in the format 'a + b'


## 3. Basic Agent with Built-in Tools

LangChain provides several built-in tools. Let's create an agent that can search the web and perform calculations:

**Optional:** To enable DuckDuckGo search functionality, install the ddgs package:
```bash
pip install -U ddgs
```

In [40]:
# Load built-in tools
# Note: Some tools may require additional API keys or installations
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Initialize Wikipedia tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Try to initialize DuckDuckGo search (requires ddgs package)
try:
    from langchain_community.tools import DuckDuckGoSearchRun
    search = DuckDuckGoSearchRun()
    tools = [wikipedia, search, get_word_length, multiply_numbers]
except ImportError:
    print("⚠️ DuckDuckGo search not available (install with: pip install -U ddgs)")
    print("Continuing with Wikipedia and custom tools only...")
    tools = [wikipedia, get_word_length, multiply_numbers]

print(f"Available tools: {[tool.name for tool in tools]}")

⚠️ DuckDuckGo search not available (install with: pip install -U ddgs)
Continuing with Wikipedia and custom tools only...
Available tools: ['wikipedia', 'get_word_length', 'multiply_numbers']


### Create an OpenAI Functions Agent

This is the recommended agent type for modern LangChain applications:

In [41]:
# Create the agent using LangGraph's create_react_agent
# This is the modern way to create agents in LangChain 1.1+
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate

# Create a system message/prompt
system_message = "You are a helpful assistant. Use the tools available to answer questions."

# Create the agent (this returns a compiled graph that acts as the agent)
agent_executor = create_react_agent(
    llm,
    tools,
    prompt=ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("placeholder", "{messages}")
    ])
)

print("✓ Agent created successfully!")

✓ Agent created successfully!


/var/folders/bm/rywd5s097fz2htlkqjrbkp440000gn/T/ipykernel_6099/3439503844.py:10: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(


### Test the Agent

Let's ask the agent some questions:

In [42]:
# Example 1: Using custom tool
response1 = agent_executor.invoke({
    "messages": [("human", "What is the length of the word 'LangChain'?")]
})
print("\n" + "="*50)
print("Answer:", response1["messages"][-1].content)


Answer: The length of the word "LangChain" is 9.


In [43]:
# Example 2: Using Wikipedia
response2 = agent_executor.invoke({
    "messages": [("human", "Who was Albert Einstein? Give me a brief summary.")]
})
print("\n" + "="*50)
print("Answer:", response2["messages"][-1].content)


Answer: Albert Einstein (1879–1955) was a German-born theoretical physicist renowned for developing the theory of relativity and contributing significantly to quantum theory. His famous equation, \( E = mc^2 \), is one of the most recognized scientific formulas. He received the Nobel Prize in Physics in 1921 for his discovery of the photoelectric effect.

Einstein studied in Switzerland, worked at the Swiss Patent Office, and later held academic positions in Berlin. He moved to the United States in 1933, fleeing Nazi persecution, and became an American citizen in 1940. He played a role in alerting the U.S. government about the potential for nuclear weapons, leading to the Manhattan Project.

Einstein's groundbreaking work includes his theories of special and general relativity, contributions to quantum mechanics, and advancements in statistical mechanics. Despite his later efforts to unify physics, he became somewhat isolated from mainstream developments in the field.


In [44]:
# Example 3: Multiple tool usage
response3 = agent_executor.invoke({
    "messages": [("human", "Search for the current population of Tokyo, then multiply that number by 2")]
})
print("\n" + "="*50)
print("Answer:", response3["messages"][-1].content)


Answer: The current population of Tokyo is approximately 14 million. When multiplied by 2, the result is 28 million.


## 4. ReAct Agent

The ReAct (Reasoning + Acting) agent uses a different prompting strategy. It's better for certain types of reasoning tasks:

In [45]:
# Create ReAct agent with custom prompt
# The ReAct agent uses a reasoning-action pattern
from langgraph.prebuilt import create_react_agent

react_prompt = ChatPromptTemplate.from_messages([
    ("system", """Answer the following questions as best you can. You have access to tools.

Use this format:
Question: the input question
Thought: think about what to do
Action: the action to take
Observation: the result
Final Answer: the final answer"""),
    ("placeholder", "{messages}")
])

react_executor = create_react_agent(
    llm,
    tools,
    prompt=react_prompt
)

# Test the ReAct agent
response = react_executor.invoke({
    "messages": [("human", "What is 25 multiplied by 4?")]
})
print("\n" + "="*50)
print("Answer:", response["messages"][-1].content)

/var/folders/bm/rywd5s097fz2htlkqjrbkp440000gn/T/ipykernel_6099/3189497329.py:17: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  react_executor = create_react_agent(



Answer: 25 multiplied by 4 is 100.


## 5. Agent with Memory

Adding memory allows the agent to remember previous interactions:

In [46]:
# Create agent with memory
# In LangGraph, memory is built-in through checkpointing
from langgraph.checkpoint.memory import MemorySaver

# Create a memory saver (checkpoint)
memory = MemorySaver()

# Create agent with memory
executor_with_memory = create_react_agent(
    llm,
    tools,
    checkpointer=memory  # This enables memory
)

print("✓ Agent with memory created!")

✓ Agent with memory created!


/var/folders/bm/rywd5s097fz2htlkqjrbkp440000gn/T/ipykernel_6099/1930139793.py:9: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  executor_with_memory = create_react_agent(


### Test Memory

Let's have a conversation with the agent:

In [47]:
# First interaction
# Create a thread/session for the conversation
config = {"configurable": {"thread_id": "conversation-1"}}

response1 = executor_with_memory.invoke({
    "messages": [("human", "My favorite number is 42.")]
}, config)
print("\n" + "="*50)
print("Response 1:", response1["messages"][-1].content)


Response 1: That's a great choice! 42 is famously known as "the answer to the ultimate question of life, the universe, and everything" in Douglas Adams' book *The Hitchhiker's Guide to the Galaxy*. Why is it your favorite?


In [48]:
# Second interaction - testing memory
# Use the same config/thread_id to maintain conversation context
response2 = executor_with_memory.invoke({
    "messages": [("human", "What is my favorite number multiplied by 2?")]
}, config)
print("\n" + "="*50)
print("Response 2:", response2["messages"][-1].content)


Response 2: Your favorite number, 42, multiplied by 2 is 84.


In [49]:
# View the conversation history
# In LangGraph, the history is stored in the state
print("\n" + "="*50)
print("Chat History (last few messages):")
for msg in response2["messages"][-4:]:  # Show last 4 messages
    print(f"\n{msg.type.upper()}: {msg.content[:100]}...")


Chat History (last few messages):

HUMAN: What is my favorite number multiplied by 2?...

AI: ...

TOOL: 84.0...

AI: Your favorite number, 42, multiplied by 2 is 84....


## 6. Practical Example: Research Assistant

Let's create a more sophisticated agent that acts as a research assistant:

In [50]:
# Create specialized tools for research
from langchain_core.tools import tool

@tool
def summarize_text(text: str) -> str:
    """Summarizes a long text into key points. Input should be the text to summarize."""
    # In a real scenario, you might use an LLM for this
    words = text.split()
    if len(words) > 50:
        return " ".join(words[:50]) + "... (truncated for brevity)"
    return text

@tool  
def save_note(note: str) -> str:
    """Saves a note or finding for later reference. Input should be the note text."""
    with open("research_notes.txt", "a") as f:
        f.write(f"\n{note}\n" + "="*50)
    return f"Note saved: {note[:50]}..."

# Create research assistant tools (reusing wikipedia and search from earlier)
# Note: Make sure you've run the earlier cells that define these tools
try:
    research_tools = [
        wikipedia,
        summarize_text,
        save_note
    ]
except NameError:
    print("⚠️ Please run the earlier cells to define 'wikipedia' tool first!")
    # Create minimal tools for demonstration
    from langchain_community.tools import WikipediaQueryRun
    from langchain_community.utilities import WikipediaAPIWrapper
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    research_tools = [wikipedia, summarize_text, save_note]

# Create the research agent
from langgraph.prebuilt import create_react_agent

research_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful research assistant. Your job is to:
    1. Search for information using available tools
    2. Summarize findings clearly
    3. Save important notes for the user
    
    Be thorough but concise in your research."""),
    ("placeholder", "{messages}")
])

research_executor = create_react_agent(
    llm,
    research_tools,
    prompt=research_prompt
)

print("✓ Research Assistant ready!")

✓ Research Assistant ready!


/var/folders/bm/rywd5s097fz2htlkqjrbkp440000gn/T/ipykernel_6099/3915414815.py:49: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  research_executor = create_react_agent(


In [51]:
# Test the research assistant
research_query = """
Research the topic of 'Quantum Computing' and provide:
1. A brief definition
2. Key applications
3. Save the most important finding as a note
"""

response = research_executor.invoke({
    "messages": [("human", research_query)]
})
print("\n" + "="*50)
print("Research Summary:")
print(response["messages"][-1].content)


Research Summary:
### 1. Definition:
Quantum computing is a field of computing that uses principles of quantum mechanics, such as superposition and entanglement, to process information. Unlike classical computers that use bits (0 or 1), quantum computers use qubits, which can exist in multiple states simultaneously.

### 2. Key Applications:
- **Cryptography**: Breaking traditional encryption methods and developing quantum-resistant cryptography.
- **Optimization**: Solving complex optimization problems in logistics, finance, and supply chain management.
- **Drug Discovery**: Simulating molecular interactions to accelerate pharmaceutical research.
- **Artificial Intelligence**: Enhancing machine learning algorithms and data analysis.
- **Material Science**: Designing new materials with specific properties.
- **Physics Simulations**: Modeling quantum systems and physical phenomena.

### 3. Saved Note:
I have saved the most important finding about quantum computing for your reference. L

## 7. Advanced: Structured Output Tools

Create tools that return structured data:

In [52]:
from typing import Dict, Any
from pydantic import BaseModel, Field
from langchain_core.tools import tool

# Define structured output using Pydantic
class PersonInfo(BaseModel):
    """Information about a person"""
    name: str = Field(description="The person's full name")
    age: int = Field(description="The person's age")
    occupation: str = Field(description="The person's occupation")

@tool
def get_person_info(name: str) -> Dict[str, Any]:
    """Get structured information about a person. Returns name, age, and occupation."""
    # This is a mock function - in reality, you'd query a database
    database = {
        "John Doe": {"name": "John Doe", "age": 30, "occupation": "Engineer"},
        "Jane Smith": {"name": "Jane Smith", "age": 25, "occupation": "Doctor"},
    }
    return database.get(name, {"name": name, "age": "Unknown", "occupation": "Unknown"})

@tool
def calculate_statistics(numbers: str) -> Dict[str, float]:
    """Calculate statistics for a list of numbers. Input should be comma-separated numbers."""
    nums = [float(x.strip()) for x in numbers.split(",")]
    return {
        "count": len(nums),
        "sum": sum(nums),
        "average": sum(nums) / len(nums),
        "min": min(nums),
        "max": max(nums)
    }

# Test structured tools
print("Person Info:", get_person_info.invoke("John Doe"))
print("Statistics:", calculate_statistics.invoke("10, 20, 30, 40, 50"))

Person Info: {'name': 'John Doe', 'age': 30, 'occupation': 'Engineer'}
Statistics: {'count': 5, 'sum': 150.0, 'average': 30.0, 'min': 10.0, 'max': 50.0}


## 8. Error Handling and Best Practices

Tips for working with agents:

In [53]:
# Best Practice 1: Use recursion limits (equivalent to max_iterations)
# LangGraph agents have built-in safety through recursion limits
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

safe_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("placeholder", "{messages}")
])

safe_executor = create_react_agent(
    llm,
    tools,
    prompt=safe_prompt
)

# When invoking, you can set a recursion limit
try:
    result = safe_executor.invoke(
        {"messages": [("human", "What is 2+2?")]},
        {"recursion_limit": 10}  # Limit the number of steps
    )
    print("✓ Executed with recursion limit")
except Exception as e:
    print(f"Error: {e}")

# Best Practice 2: Good tool descriptions
@tool
def well_documented_tool(query: str) -> str:
    """
    This tool searches for information about a topic.
    
    Args:
        query: The search query. Should be a clear, specific question or topic.
        
    Returns:
        A string containing relevant information about the query.
        
    Examples:
        - "What is machine learning?"
        - "History of the internet"
    """
    return f"Mock result for: {query}"

print("\n✓ Best practices demonstrated!")
print("\nKey Takeaways:")
print("1. Use recursion_limit in config to prevent infinite loops")
print("2. LangGraph agents have built-in error handling")
print("3. Write clear, detailed tool descriptions")
print("4. Test tools individually before adding to agents")
print("5. Use checkpointers for memory/state management")

/var/folders/bm/rywd5s097fz2htlkqjrbkp440000gn/T/ipykernel_6099/3866118375.py:11: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  safe_executor = create_react_agent(


✓ Executed with recursion limit

✓ Best practices demonstrated!

Key Takeaways:
1. Use recursion_limit in config to prevent infinite loops
2. LangGraph agents have built-in error handling
3. Write clear, detailed tool descriptions
4. Test tools individually before adding to agents
5. Use checkpointers for memory/state management


## 9. Agent Types Comparison

LangChain offers several agent types. Here's when to use each:

### Modern Agent Architecture (LangChain 1.1+):

**LangGraph's create_react_agent** is now the recommended approach:
- Built on LangGraph for better state management
- Supports memory through checkpointers
- More flexible and composable
- Better error handling and debugging

### Key Concepts:

1. **create_react_agent** (Recommended)
   - Modern approach using LangGraph
   - Works with all OpenAI models
   - Built-in state management
   - Supports checkpointing for memory

2. **State Management**
   - Messages are stored in the agent's state
   - Checkpointers enable persistent memory
   - Thread IDs manage separate conversations

3. **Customization**
   - Use `state_modifier` for system prompts
   - Pass custom prompts from LangChain Hub
   - Control behavior through configuration

### When to Use:

- **For most applications**: Use `create_react_agent` from LangGraph
- **Custom workflows**: Build custom graphs with LangGraph
- **Simple tasks**: Use LLM chains without agents
- **Complex reasoning**: Add more sophisticated graph structures

## 10. Real-World Use Cases

Common applications of LangChain agents:

### 1. **Customer Support Bot**
- Tools: Database lookup, ticket creation, FAQ search
- Use case: Answer questions, create support tickets, escalate issues

### 2. **Data Analysis Assistant**
- Tools: SQL queries, data visualization, statistical analysis
- Use case: Query databases, generate reports, create charts

### 3. **Research Assistant**
- Tools: Web search, paper lookup, summarization
- Use case: Gather information, summarize findings, cite sources

### 4. **Code Helper**
- Tools: Code search, documentation lookup, code execution
- Use case: Answer coding questions, debug issues, suggest solutions

### 5. **Personal Assistant**
- Tools: Calendar, email, reminders, weather
- Use case: Schedule meetings, send emails, set reminders

### 6. **Content Creator**
- Tools: Research, writing, image generation, formatting
- Use case: Research topics, write articles, generate images

### 7. **Financial Advisor**
- Tools: Stock data, news, calculations, portfolio analysis
- Use case: Analyze stocks, track portfolio, provide insights

## 11. Summary and Key Concepts

### What We Learned:

1. **Modern Agents with LangGraph**: Agents in LangChain 1.1+ use LangGraph for better state management

2. **Core Components**:
   - **LLM**: The brain making decisions (e.g., ChatOpenAI)
   - **Tools**: Functions the agent can use
   - **create_react_agent**: Creates an agent graph
   - **Checkpointer**: Enables memory/state persistence
   - **Config**: Controls execution (thread_id, recursion_limit)

3. **Creating Tools**:
   - Use `@tool` decorator for simple tools
   - Provide clear descriptions with examples
   - Test tools independently first

4. **Agent Invocation**:
   - Use `.invoke()` with messages format
   - Pass config for memory (thread_id)
   - Set recursion_limit for safety

5. **Best Practices**:
   - Set `recursion_limit` in config
   - Write detailed tool descriptions
   - Test tools individually
   - Use checkpointers for memory
   - Handle API keys securely

### Modern LangChain Architecture:

```python
from langgraph.prebuilt import create_react_agent

# Create agent
agent = create_react_agent(llm, tools)

# Invoke agent
result = agent.invoke(
    {"messages": [("human", "your question")]},
    {"recursion_limit": 10}
)
```

### Next Steps:

- Experiment with different tool combinations
- Build custom LangGraph workflows
- Add persistent memory with checkpointers
- Integrate with external APIs
- Explore streaming responses
- Deploy agents in production

### Resources:

- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [LangChain Documentation](https://python.langchain.com/)
- [Tool Documentation](https://python.langchain.com/docs/modules/agents/tools/)
- [LangChain Hub](https://smith.langchain.com/hub)

## Practice Exercises

Try these exercises to deepen your understanding:

### Exercise 1: Calculator Agent
Create an agent with tools for basic math operations (add, subtract, multiply, divide).

### Exercise 2: Weather Agent
Build an agent that can fetch weather data for different cities (you'll need a weather API).

### Exercise 3: File Manager Agent
Create an agent with tools to list, read, and search through files in a directory.

### Exercise 4: Multi-Agent System
Build two agents that can collaborate - one for research, one for summarization.

### Exercise 5: Custom Tool
Create a tool that interacts with a database or API of your choice.

---

**Congratulations!** You now understand how to work with LangChain Agents! 🎉